In [ ]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
!pip install torchsummary


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(1, 10, 3, padding = 'valid') # 28*28   RF = 3*3
        self.norm1 = nn.BatchNorm2d(10)  
        self.conv2 = nn.Conv2d(10, 20, 3, padding = 'valid') #26*26*20  RF = 5*5
        self.norm2 = nn.BatchNorm2d(20)
        self.pool1 = nn.MaxPool2d(2, 2)  #24*24*20  RF = 10*10 
        
        self.conv3 = nn.Conv2d(20, 20, 3, padding = 'valid')  #12*12*20  RF = 12*12
        self.norm3 = nn.BatchNorm2d(20)
        self.conv4 = nn.Conv2d(20,20,3, padding = 'same')   # 10*10*10  RF = 14*14
        self.pool2 = nn.MaxPool2d(2,2)  # 5*5*10     RF = 28*28
        
        self.conv5 = nn.Conv2d(20,20, 1, padding = 'valid')  #5*5*10   RF = 28*28
        self.conv6 = nn.AvgPool2d(5)    # 5*5*10  RF = 32*32
        self.fc1   = nn.Linear(20,10)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.1)

    def forward(self, x):
        x = self.dropout(self.norm1(self.relu(self.conv1(x))))
        x = self.dropout(self.pool1(self.norm2(self.relu(self.conv2(x)))))
        x = self.dropout(self.pool2(self.relu(self.conv4(self.norm3(self.relu(self.conv3(x)))))))
        x = self.conv6(self.relu(self.conv5(x)))
        x = x.view(-1, 20)
        x = self.fc1(x)
        # x = self.fc1(x)
        

        return F.log_softmax(x, dim = 1)

In [ ]:
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu") 
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 10, 26, 26]             100
              ReLU-2           [-1, 10, 26, 26]               0
       BatchNorm2d-3           [-1, 10, 26, 26]              20
           Dropout-4           [-1, 10, 26, 26]               0
            Conv2d-5           [-1, 20, 24, 24]           1,820
              ReLU-6           [-1, 20, 24, 24]               0
       BatchNorm2d-7           [-1, 20, 24, 24]              40
         MaxPool2d-8           [-1, 20, 12, 12]               0
           Dropout-9           [-1, 20, 12, 12]               0
           Conv2d-10           [-1, 20, 10, 10]           3,620
             ReLU-11           [-1, 20, 10, 10]               0
      BatchNorm2d-12           [-1, 20, 10, 10]              40
           Conv2d-13           [-1, 20, 10, 10]           3,620
             ReLU-14           [-1, 20,

In [ ]:


torch.manual_seed(1)
batch_size = 256

kwargs = {'num_workers': 6, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
from tqdm import tqdm

train_losses = []
test_losses = []
train_acc = []
test_acc = []

def train(model, device, train_loader, optimizer, epoch):
  model.train()
  pbar = tqdm(train_loader)
  correct = 0
  processed = 0
  for batch_idx, (data, target) in enumerate(pbar):
    # get samples
    data, target = data.to(device), target.to(device)

    # Init
    optimizer.zero_grad()
    # In PyTorch, we need to set the gradients to zero before starting to do backpropragation because PyTorch accumulates the gradients on subsequent backward passes. 
    # Because of this, when you start your training loop, ideally you should zero out the gradients so that you do the parameter update correctly.

    # Predict
    y_pred = model(data)

    # Calculate loss
    loss = F.nll_loss(y_pred, target)
    train_losses.append(loss)

    # Backpropagation
    loss.backward()
    optimizer.step()

    # Update pbar-tqdm
    
    pred = y_pred.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
    correct += pred.eq(target.view_as(pred)).sum().item()
    processed += len(data)

    pbar.set_description(desc= f'Loss={loss.item()} Batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')
    train_acc.append(100*correct/processed)

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
    test_acc.append(100. * correct / len(test_loader.dataset))

In [ ]:
model =  Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
EPOCHS = 15
for epoch in range(EPOCHS):
    print("EPOCH:", epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

EPOCH: 0


Loss=0.8718722462654114 Batch_id=234 Accuracy=39.41: 100%|██████████| 235/235 [00:14<00:00, 16.19it/s]



Test set: Average loss: 1.2783, Accuracy: 5167/10000 (51.67%)

EPOCH: 1


Loss=0.36924317479133606 Batch_id=234 Accuracy=83.70: 100%|██████████| 235/235 [00:13<00:00, 17.91it/s]



Test set: Average loss: 0.3972, Accuracy: 8694/10000 (86.94%)

EPOCH: 2


...


Test set: Average loss: 0.0491, Accuracy: 9846/10000 (98.46%)

EPOCH: 13


Loss=0.053033843636512756 Batch_id=234 Accuracy=98.07: 100%|██████████| 235/235 [00:12<00:00, 18.26it/s]



Test set: Average loss: 0.0487, Accuracy: 9852/10000 (98.52%)

EPOCH: 14


Loss=0.052106499671936035 Batch_id=234 Accuracy=98.20: 100%|██████████| 235/235 [00:14<00:00, 16.39it/s]



Test set: Average loss: 0.0485, Accuracy: 9859/10000 (98.59%)

